In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from lightgbm import LGBMRegressor
from mlflow import log_metric, log_param, log_artifacts
import mlflow
import mlflow.lightgbm
import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit

/Users/rgarzon/anaconda3/envs/vinya/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
ss = pd.read_csv('./content/SampleSubmission_ZxsCq0O.csv')
train = pd.read_csv('./content/Train_awoL0xl.csv')
test = pd.read_csv('./content/Test_QQKW4dv.csv')


In [26]:
ID_COL,TARGET_COL = 'ID','Close'

In [27]:
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
  return mean_squared_error(y_true, y_pred) ** 0.5

In [28]:
def download_preds(preds_test, file_name = 'hacklive_sub.csv'):

  ## 1. Setting the target column with our obtained predictions
  ss[TARGET_COL] = preds_test

  ## 2. Saving our predictions to a csv file

  ss.to_csv(file_name, index = False)

  ## 3. Downloading and submitting the csv file
#  from google.colab import files
#  files.download(file_name)

In [29]:
df = pd.concat([train,test],axis=0).reset_index(drop=True)

In [30]:
df['Date'] = pd.to_datetime(df['Date'])
for attr in ['year','month','day','week','dayofweek']:
    df[attr] = getattr(df['Date'].dt,attr)

/Users/rgarzon/anaconda3/envs/vinya/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
df['prev_1'] = df.iloc[:,-2] #Here -2 as -1 is a target

In [32]:
for win in [2, 3, 5, 7, 10, 14, 21, 28, 56]:
    tmp = df.iloc[:,-1-win:-1]
    #General statistics for base level
    df['mean_prev_{}'.format(win)] = tmp.mean(axis=1)
    df['median_prev_{}'.format(win)] = tmp.median(axis=1)
    df['min_prev_{}'.format(win)] = tmp.min(axis=1)
    df['max_prev_{}'.format(win)] = tmp.max(axis=1)
    df['std_prev_{}'.format(win)] = tmp.std(axis=1)
    #Capturing trend
    df['mean_ewm_prev_{}'.format(win)] = tmp.T.ewm(com=9.5).mean().T.mean(axis=1)
    df['last_ewm_prev_{}'.format(win)] = tmp.T.ewm(com=9.5).mean().T.iloc[:,-1]
    
    df['avg_diff_{}'.format(win)] = (tmp - tmp.shift(1, axis=1)).mean(axis=1)
    df['avg_div_{}'.format(win)] =  (tmp / tmp.shift(1, axis=1)).mean(axis=1)

In [33]:
for win in [2, 3, 4, 8]:
    tmp = df.iloc[:,-1-win*7:-1:7] #7 for week
    #Features for weekly seasonality
    df['week_mean_prev_{}'.format(win)] = tmp.mean(axis=1)
    df['week_median_prev_{}'.format(win)] = tmp.median(axis=1)
    df['week_min_prev_{}'.format(win)] = tmp.min(axis=1)
    df['week_max_prev_{}'.format(win)] = tmp.max(axis=1)
    df['week_std_prev_{}'.format(win)] = tmp.std(axis=1)

In [34]:
train,test = df[:train.shape[0]].reset_index(drop=True),df[train.shape[0]:].reset_index(drop=True)

In [35]:
features = ['stock','holiday', 'unpredictability_score', 'year','month','day','week','dayofweek']

In [36]:
features = ['stock','holiday', 'unpredictability_score', 'year','month','day','week','dayofweek',
            'unpredictability_score', 'prev_1', 'mean_prev_2', 'median_prev_2',
       'min_prev_2', 'max_prev_2', 'std_prev_2', 'mean_ewm_prev_2',
       'last_ewm_prev_2', 'avg_diff_2', 'avg_div_2', 'mean_prev_3',
       'median_prev_3', 'min_prev_3', 'max_prev_3', 'std_prev_3',
       'mean_ewm_prev_3', 'last_ewm_prev_3', 'avg_diff_3', 'avg_div_3',
       'mean_prev_5', 'median_prev_5', 'min_prev_5', 'max_prev_5',
       'std_prev_5', 'mean_ewm_prev_5', 'last_ewm_prev_5', 'avg_diff_5',
       'avg_div_5', 'mean_prev_7', 'median_prev_7', 'min_prev_7', 'max_prev_7',
       'std_prev_7', 'mean_ewm_prev_7', 'last_ewm_prev_7', 'avg_diff_7',
       'avg_div_7', 'mean_prev_10', 'median_prev_10', 'min_prev_10',
       'max_prev_10', 'std_prev_10', 'mean_ewm_prev_10', 'last_ewm_prev_10',
       'avg_diff_10', 'avg_div_10', 'mean_prev_14', 'median_prev_14',
       'min_prev_14', 'max_prev_14', 'std_prev_14', 'mean_ewm_prev_14',
       'last_ewm_prev_14', 'avg_diff_14', 'avg_div_14', 'mean_prev_21',
       'median_prev_21', 'min_prev_21', 'max_prev_21', 'std_prev_21',
       'mean_ewm_prev_21', 'last_ewm_prev_21', 'avg_diff_21', 'avg_div_21',
       'mean_prev_28', 'median_prev_28', 'min_prev_28', 'max_prev_28',
       'std_prev_28', 'mean_ewm_prev_28', 'last_ewm_prev_28', 'avg_diff_28',
       'avg_div_28', 'mean_prev_56', 'median_prev_56', 'min_prev_56',
       'max_prev_56', 'std_prev_56', 'mean_ewm_prev_56', 'last_ewm_prev_56',
       'avg_diff_56', 'avg_div_56', 'week_mean_prev_2', 'week_median_prev_2',
       'week_min_prev_2', 'week_max_prev_2', 'week_std_prev_2',
       'week_mean_prev_3', 'week_median_prev_3', 'week_min_prev_3',
       'week_max_prev_3', 'week_std_prev_3', 'week_mean_prev_4',
       'week_median_prev_4', 'week_min_prev_4', 'week_max_prev_4',
       'week_std_prev_4', 'week_mean_prev_8', 'week_median_prev_8',
       'week_min_prev_8', 'week_max_prev_8', 'week_std_prev_8']

In [37]:
FIRST_TRAINING_DAY = train['Date'].min()
DAYS_TO_VALIDATE = pd.Timedelta(days=200)
VAL_LAST_DAY = FIRST_TRAINING_DAY + DAYS_TO_VALIDATE
VAL_LAST_DAY - FIRST_TRAINING_DAY

val_fltr = train['Date']<= VAL_LAST_DAY
trn,val = train[~val_fltr].reset_index(drop=True),train[val_fltr].reset_index(drop=True)

In [38]:
clf = LGBMRegressor (n_estimators=30000)

In [39]:
clf.fit(train[features],train['Close'],eval_set=[(val[features],val[TARGET_COL])],verbose=50 ,early_stopping_rounds=200, eval_metric='rmse')

Training until validation scores don't improve for 200 rounds
[50]	valid_0's rmse: 6.43471	valid_0's l2: 41.4055
[100]	valid_0's rmse: 3.80129	valid_0's l2: 14.4498
[150]	valid_0's rmse: 2.98742	valid_0's l2: 8.92468
[200]	valid_0's rmse: 2.60904	valid_0's l2: 6.80709
[250]	valid_0's rmse: 2.3193	valid_0's l2: 5.37917
[300]	valid_0's rmse: 2.04812	valid_0's l2: 4.19478
[350]	valid_0's rmse: 1.85418	valid_0's l2: 3.43799
[400]	valid_0's rmse: 1.74345	valid_0's l2: 3.03961
[450]	valid_0's rmse: 1.66597	valid_0's l2: 2.77546
[500]	valid_0's rmse: 1.59998	valid_0's l2: 2.55994
[550]	valid_0's rmse: 1.54522	valid_0's l2: 2.3877
[600]	valid_0's rmse: 1.49699	valid_0's l2: 2.24098
[650]	valid_0's rmse: 1.40981	valid_0's l2: 1.98756
[700]	valid_0's rmse: 1.35847	valid_0's l2: 1.84545
[750]	valid_0's rmse: 1.28097	valid_0's l2: 1.64087
[800]	valid_0's rmse: 1.25718	valid_0's l2: 1.58051
[850]	valid_0's rmse: 1.21588	valid_0's l2: 1.47836
[900]	valid_0's rmse: 1.18221	valid_0's l2: 1.39762
[950]

[7550]	valid_0's rmse: 0.431245	valid_0's l2: 0.185973
[7600]	valid_0's rmse: 0.430421	valid_0's l2: 0.185262
[7650]	valid_0's rmse: 0.429514	valid_0's l2: 0.184482
[7700]	valid_0's rmse: 0.427627	valid_0's l2: 0.182865
[7750]	valid_0's rmse: 0.426218	valid_0's l2: 0.181661
[7800]	valid_0's rmse: 0.425313	valid_0's l2: 0.180891
[7850]	valid_0's rmse: 0.424133	valid_0's l2: 0.179889
[7900]	valid_0's rmse: 0.422165	valid_0's l2: 0.178224
[7950]	valid_0's rmse: 0.420955	valid_0's l2: 0.177203
[8000]	valid_0's rmse: 0.420169	valid_0's l2: 0.176542
[8050]	valid_0's rmse: 0.417732	valid_0's l2: 0.1745
[8100]	valid_0's rmse: 0.416279	valid_0's l2: 0.173289
[8150]	valid_0's rmse: 0.415189	valid_0's l2: 0.172382
[8200]	valid_0's rmse: 0.412508	valid_0's l2: 0.170163
[8250]	valid_0's rmse: 0.411054	valid_0's l2: 0.168965
[8300]	valid_0's rmse: 0.409398	valid_0's l2: 0.167607
[8350]	valid_0's rmse: 0.408284	valid_0's l2: 0.166696
[8400]	valid_0's rmse: 0.405468	valid_0's l2: 0.164404
[8450]	valid

[14950]	valid_0's rmse: 0.298927	valid_0's l2: 0.0893574
[15000]	valid_0's rmse: 0.298352	valid_0's l2: 0.0890139
[15050]	valid_0's rmse: 0.297808	valid_0's l2: 0.0886898
[15100]	valid_0's rmse: 0.297298	valid_0's l2: 0.0883863
[15150]	valid_0's rmse: 0.296865	valid_0's l2: 0.0881287
[15200]	valid_0's rmse: 0.296457	valid_0's l2: 0.0878869
[15250]	valid_0's rmse: 0.295845	valid_0's l2: 0.0875242
[15300]	valid_0's rmse: 0.294899	valid_0's l2: 0.0869655
[15350]	valid_0's rmse: 0.294633	valid_0's l2: 0.0868083
[15400]	valid_0's rmse: 0.294227	valid_0's l2: 0.0865696
[15450]	valid_0's rmse: 0.293364	valid_0's l2: 0.0860624
[15500]	valid_0's rmse: 0.292609	valid_0's l2: 0.0856199
[15550]	valid_0's rmse: 0.292256	valid_0's l2: 0.0854134
[15600]	valid_0's rmse: 0.291485	valid_0's l2: 0.0849637
[15650]	valid_0's rmse: 0.29032	valid_0's l2: 0.0842856
[15700]	valid_0's rmse: 0.289844	valid_0's l2: 0.0840093
[15750]	valid_0's rmse: 0.288571	valid_0's l2: 0.0832735
[15800]	valid_0's rmse: 0.287797

[22200]	valid_0's rmse: 0.233782	valid_0's l2: 0.0546538
[22250]	valid_0's rmse: 0.233483	valid_0's l2: 0.0545141
[22300]	valid_0's rmse: 0.233018	valid_0's l2: 0.0542973
[22350]	valid_0's rmse: 0.232862	valid_0's l2: 0.0542248
[22400]	valid_0's rmse: 0.232574	valid_0's l2: 0.0540908
[22450]	valid_0's rmse: 0.232275	valid_0's l2: 0.0539517
[22500]	valid_0's rmse: 0.231898	valid_0's l2: 0.0537766
[22550]	valid_0's rmse: 0.231472	valid_0's l2: 0.0535793
[22600]	valid_0's rmse: 0.231005	valid_0's l2: 0.0533635
[22650]	valid_0's rmse: 0.230681	valid_0's l2: 0.0532138
[22700]	valid_0's rmse: 0.230238	valid_0's l2: 0.0530097
[22750]	valid_0's rmse: 0.229937	valid_0's l2: 0.0528712
[22800]	valid_0's rmse: 0.229656	valid_0's l2: 0.0527417
[22850]	valid_0's rmse: 0.229477	valid_0's l2: 0.0526599
[22900]	valid_0's rmse: 0.229286	valid_0's l2: 0.0525723
[22950]	valid_0's rmse: 0.229075	valid_0's l2: 0.0524754
[23000]	valid_0's rmse: 0.22884	valid_0's l2: 0.0523675
[23050]	valid_0's rmse: 0.22843	

[29450]	valid_0's rmse: 0.190084	valid_0's l2: 0.036132
[29500]	valid_0's rmse: 0.189958	valid_0's l2: 0.0360839
[29550]	valid_0's rmse: 0.18989	valid_0's l2: 0.0360583
[29600]	valid_0's rmse: 0.189767	valid_0's l2: 0.0360114
[29650]	valid_0's rmse: 0.189571	valid_0's l2: 0.035937
[29700]	valid_0's rmse: 0.189332	valid_0's l2: 0.0358464
[29750]	valid_0's rmse: 0.18918	valid_0's l2: 0.0357892
[29800]	valid_0's rmse: 0.189023	valid_0's l2: 0.0357298
[29850]	valid_0's rmse: 0.188952	valid_0's l2: 0.0357028
[29900]	valid_0's rmse: 0.188848	valid_0's l2: 0.0356637
[29950]	valid_0's rmse: 0.188673	valid_0's l2: 0.0355976
[30000]	valid_0's rmse: 0.188542	valid_0's l2: 0.035548
Did not meet early stopping. Best iteration is:
[30000]	valid_0's rmse: 0.188542	valid_0's l2: 0.035548


LGBMRegressor(n_estimators=30000)

In [40]:
with mlflow.start_run():
    rmse_eval = rmse (val[TARGET_COL],clf.predict(val[features]))
    log_param("rmse", rmse_eval)
    log_param("n_estimators",30000)
    log_param("algorithm", "Full dataset, validation first 200 days")
    pt = clf.predict(test[features])

In [41]:
test['lightgbm'] = pt

/Users/rgarzon/anaconda3/envs/vinya/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
from fbprophet import Prophet

for one_stock in range(103):
    print (one_stock)
    d = {'y': train[train['stock']==one_stock]['Close'],'ds' : train[train['stock']==one_stock]['Date']}
    df = pd.DataFrame(d)   
    model = Prophet()
    model.fit(df)                
    d = {'ds' : test[test['stock']==one_stock]['Date']}
    future = pd.DataFrame(d)
    forecast = model.predict(future)
    #print (forecast['yhat'])
    test.loc[test['stock']==one_stock,'fbprophet'] = forecast['yhat'].values
    #print (test[test['stock']==one_stock]['Close'])
    #print (test[test['stock']==one_stock].head)
    print ("************************")

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


************************
1


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

************************
100


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


************************
101


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


************************
102
************************


In [43]:
print (test.columns)
print (test['lightgbm'].describe)
print (test['fbprophet'].describe)

Index(['ID', 'stock', 'Date', 'Open', 'High', 'Low', 'Close', 'holiday',
       'unpredictability_score', 'year',
       ...
       'week_min_prev_4', 'week_max_prev_4', 'week_std_prev_4',
       'week_mean_prev_8', 'week_median_prev_8', 'week_min_prev_8',
       'week_max_prev_8', 'week_std_prev_8', 'lightgbm', 'fbprophet'],
      dtype='object', length=118)
<bound method NDFrame.describe of 0       116.207541
1       116.399150
2       117.404643
3       117.871411
4       118.260974
           ...    
4218    129.577622
4219    125.935722
4220    126.774561
4221    130.922788
4222    130.023494
Name: lightgbm, Length: 4223, dtype: float64>
<bound method NDFrame.describe of 0       119.868441
1       120.542837
2       120.903800
3       121.169006
4       121.551232
           ...    
4218    133.068740
4219    133.083370
4220    133.297479
4221    133.411664
4222    133.486280
Name: fbprophet, Length: 4223, dtype: float64>


/Users/rgarzon/anaconda3/envs/vinya/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
test['Close'] = (test['lightgbm']+test['fbprophet'])/2

In [45]:
print(test['Close'].isna().sum())
test['Close'].head

0


<bound method NDFrame.head of 0       118.037991
1       118.470993
2       119.154222
3       119.520208
4       119.906103
           ...    
4218    131.323181
4219    129.509546
4220    130.036020
4221    132.167226
4222    131.754887
Name: Close, Length: 4223, dtype: float64>

In [46]:
download_preds(test['Close'],'lightgbm_1ba474ee67e94b81acd97f80925fd051_v2.csv')